# Week 5: Neural Network

### What's on this week
1. [Resuming from week 4](#resume)
2. [Building your first neural network model](#build)
3. [Understanding your neural network model](#viz)
4. [Finding optimal hyperparameters with GridSearchCV](#gridsearch)
5. [Feature selection](#fselect)
6. [Comparing each model](#comparison)

---

The practical note for this week introduces you to neural network mining in Python, particularly using multilayer perceptron classifier. Neural networks are a class of predictive models that mimic the structure of human brain. It consists of layers of neurons, each consuming outputs from the previous layers as inputs. Neural network is the most complex model out of everything that we have used so far.

**This tutorial notes is in experimental version. Please give us feedbacks and suggestions on how to make it better. Ask your tutor for any question and clarification.**

## 1. Resuming from week 4<a name="resume"></a>
Last week, we learned how to perform data mining with decision trees in Python. For this week, we will again reuse the code for data preprocessing. Just as regression models, neural networks are sensitive to skewed data, thus we also perform standarization on it:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from dm_tools import data_prep
from sklearn.preprocessing import StandardScaler

# preprocessing step
df = data_prep()

# train test split
y = df['TargetB']
X = df.drop(['TargetB'], axis=1)
X_mat = X.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.5, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train, y_train)
X_test = scaler.transform(X_test)